In [57]:
#Structuring the data
import numpy as np
import pandas as pd
import json
import matplotlib.pyplot as plt
from datetime import datetime

In [88]:
receipts = pd.read_json('receipts.json', lines=True)
receipts['_id'] = receipts['_id'].apply(lambda x: x['$oid'])
def DateTime(df, col):
  list=[]
  for x in df[col].values:
    if type(x) == float:
      list.append(np.nan)
    else:
      list.append(pd.to_datetime(datetime.fromtimestamp(x['$date'] // 1000)))
  df[col] = list
  return df
date_cols = ['createDate', 'dateScanned', 'finishedDate',
             'modifyDate', 'pointsAwardedDate', 'purchaseDate']
for col in date_cols:
  receipts = DateTime(receipts, col)

In [89]:

users = pd.read_json('users.json', lines=True)


users['_id'] = users['_id'].apply(lambda x: x['$oid'])


date_cols = ['createdDate', 'lastLogin']
for col in date_cols:
  users = DateTime(users, col)

In [90]:

brands = pd.read_json('brands.json', lines=True)


brands['_id'] = brands['_id'].apply(lambda x: x['$oid'])

In [91]:

cpg = pd.DataFrame()
id=[]
oid=[]
ref=[]

for i in range(len(brands['cpg'])):
  x = brands['cpg'].iloc[i]
  if type(x) != float:
      oid.append(brands['cpg'][i]['$id']['$oid'])
      ref.append(brands['cpg'][i]['$ref'])
      id.append(brands.iloc[i]['_id'])

cpg['id'] = id
cpg['oid'] = oid
cpg['ref'] = ref

In [93]:
brands = brands.merge(cpg, left_on='_id', right_on='id', how='left').drop('id', axis=1)

In [94]:
#Finding Null Values to detect quality issues
receipts.isna().sum()
users.isna().sum()
brands.isna().sum()

_id               0
barcode           0
category        155
categoryCode    650
cpg               0
name              0
topBrand        612
brandCode       234
oid_x             0
ref_x             0
oid_y             0
ref_y             0
dtype: int64

In [72]:
# What are the top 5 brands by receipts scanned for most recent month?

#SELECT CASE 	WHEN  		( SELECT avg( totalSpent ) FROM receipts WHERE rewardsReceiptStatus = 'FINISHED' )  		>  		( SELECT avg( totalSpent ) FROM receipts WHERE rewardsReceiptStatus = 'REJECTED' ) 	THEN "Average spend from receipts with 'rewardsReceiptStatus’ of ‘Accepted’ is greater"         WHEN  		( SELECT avg( totalSpent ) FROM receipts WHERE rewardsReceiptStatus = 'FINISHED' )  		<  		( SELECT avg( totalSpent ) FROM receipts WHERE rewardsReceiptStatus = 'REJECTED' )         THEN "Average spend from receipts with 'rewardsReceiptStatus’ of ‘Rejected’ is greater" 	ELSE "They are the same" 	END AS COMPARING ;